Setting up the kaggle Api


In [ ]:
!pip install kaggle
import kaggle

In [3]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv  #for fetching the dataset from kaggle.com

usage: kaggle [-h] [-v]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...
kaggle: error: unrecognized arguments: #for fetching the dataset from kaggle.com


In [4]:
#extract file from zip file
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip') 
zip_ref.extractall() 
zip_ref.close() 

Importing the Data Set

In [5]:
import pandas as pd

In [6]:
df=pd.read_csv(r"C:\Users\dell\Desktop\Project\orders.csv")

Quick EDA



In [7]:
df.isnull().sum()

Order Id            0
Order Date          0
Ship Mode           1
Segment             0
Country             0
City                0
State               0
Postal Code         0
Region              0
Category            0
Sub Category        0
Product Id          0
cost price          0
List Price          0
Quantity            0
Discount Percent    0
dtype: int64

In [8]:
df['Ship Mode'].value_counts() #as we can see its an categorical column

Ship Mode
Standard Class    5962
Second Class      1945
First Class       1538
Same Day           543
Not Available        4
unknown              1
Name: count, dtype: int64

In [9]:
df['Ship Mode'].fillna(df['Ship Mode'].mode().iloc[0],inplace=True)

In [10]:
df.loc[(df['Ship Mode']=='Not Available')|(df['Ship Mode']=='unknown'),'Ship Mode']=df['Ship Mode'].mode().iloc[0]

In [11]:
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', 'First Class', 'Same Day'],
      dtype=object)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Order Id          9994 non-null   int64 
 1   Order Date        9994 non-null   object
 2   Ship Mode         9994 non-null   object
 3   Segment           9994 non-null   object
 4   Country           9994 non-null   object
 5   City              9994 non-null   object
 6   State             9994 non-null   object
 7   Postal Code       9994 non-null   int64 
 8   Region            9994 non-null   object
 9   Category          9994 non-null   object
 10  Sub Category      9994 non-null   object
 11  Product Id        9994 non-null   object
 12  cost price        9994 non-null   int64 
 13  List Price        9994 non-null   int64 
 14  Quantity          9994 non-null   int64 
 15  Discount Percent  9994 non-null   int64 
dtypes: int64(6), object(10)
memory usage: 1.2+ MB


In [13]:
df['Order Date']

0       2023-03-01
1       2023-08-15
2       2023-01-10
3       2022-06-18
4       2022-07-13
           ...    
9989    2023-02-18
9990    2023-03-17
9991    2022-08-07
9992    2022-11-19
9993    2022-07-17
Name: Order Date, Length: 9994, dtype: object

In [14]:
df['Order Date']=pd.to_datetime(df['Order Date'],format='%Y-%m-%d')

In [15]:
df.dtypes

Order Id                     int64
Order Date          datetime64[ns]
Ship Mode                   object
Segment                     object
Country                     object
City                        object
State                       object
Postal Code                  int64
Region                      object
Category                    object
Sub Category                object
Product Id                  object
cost price                   int64
List Price                   int64
Quantity                     int64
Discount Percent             int64
dtype: object

#Converting all columns to small case and replace the space to '_' (method-1)


In [16]:
df.columns  #because in future we have to analyse the data in SQL

Index(['Order Id', 'Order Date', 'Ship Mode', 'Segment', 'Country', 'City',
       'State', 'Postal Code', 'Region', 'Category', 'Sub Category',
       'Product Id', 'cost price', 'List Price', 'Quantity',
       'Discount Percent'],
      dtype='object')

In [17]:
# a=[]
# b=[]
# d={}
# for n in df.columns:
#     a.append(n)

# for n in a:
#     p=n.lower()
#     q=p.split()
#     r='_'.join(q)
#     b.append(r)
    
# for n in a:
#     d[n]=b[a.index(n)]
# d


In [18]:
# df.rename(columns=d,inplace=True)

Pandas string functon method(2)

In [19]:
# df.columns
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ','_')
df.head(1)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2


Derived "discount" Column 

In [20]:
# 260*2/100
df['discount']=df['list_price']*df['discount_percent']/100

In [21]:
df.head(1)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2


Derived "sale_price" Column 

In [22]:
df['sale_price']=df['list_price']-df['discount']

Derived "profit" column

In [23]:
df['profit']=df['sale_price']-df['cost_price']

In [24]:
df.drop(['cost_price','list_price','discount_percent'],axis=1,inplace=True)

In [25]:
df

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,2023-02-18,Second Class,Consumer,United States,Miami,Florida,33180,South,Furniture,Furnishings,FUR-FU-10001889,3,1.2,28.8,-1.2
9990,9991,2023-03-17,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Furniture,Furnishings,FUR-FU-10000747,2,3.6,86.4,16.4
9991,9992,2022-08-07,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Technology,Phones,TEC-PH-10003645,2,5.2,254.8,34.8
9992,9993,2022-11-19,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Office Supplies,Paper,OFF-PA-10004041,4,0.9,29.1,-0.9


In [26]:
df.to_csv("retail_order_clean.csv",index=False)

In [30]:
df['sub_category'].value_counts()

sub_category
Binders        1523
Paper          1370
Furnishings     957
Phones          889
Storage         846
Art             796
Accessories     775
Chairs          617
Appliances      466
Labels          364
Tables          319
Envelopes       254
Bookcases       228
Fasteners       217
Supplies        190
Machines        115
Copiers          68
Name: count, dtype: int64